# Oanda Demo Trading Notebook

## Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import datetime
import winsound
import collections

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

import oandapyV20.endpoints.orders as orders

from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import oandapyV20.endpoints.positions as positions

Custom packages

In [3]:
from utils.toolsapi import get_date_time
from utils.toolsapi import get_prices
from utils.toolsapi import calc_duration

## Functions

In [4]:
def get_targets(start_price, target_num = 3):
    positive_targets = {}
    negative_targets = {}
    pip = 10**-4
    
    for i in range(target_num):
        move_val = pip*(i+1)
        positive_targets[i] = start_price + move_val
        negative_targets[i] = start_price - move_val
        
    return(positive_targets, negative_targets)

In [5]:
def get_direction(df_reached_targets, target_num,  min_count = 10):
    tot_pos = sum(df_reached_targets['positive'] * df_reached_targets['target_num'])
    tot_neg = sum(df_reached_targets['negative'] * df_reached_targets['target_num'])

    if ((tot_pos - tot_neg) / target_num) > 1 and tot_pos > min_count and df_reached_targets['positive'][1] > 0:
        direction = 'positive'
    elif ((tot_neg - tot_pos) / target_num) > 1 and tot_neg > min_count and df_reached_targets['negative'][1] > 0:
        direction = 'negative'
    else:
        direction = 'no_direction'
    return(direction)

In [6]:
def run_direction_engine(instrument, num, target_num, min_count): 
    params = {'instruments': instrument}
    
    reached_targets = {'start_price' : 0,
                       'target_num' : 0,
                       'positive_val' : 0,
                       'negative_val' : 0,
                       'positive' : 0,
                       'p_duration' : '',
                       'n_duration' : '',
                       'negative' : 0}

    pos_target_flag = 'not_reached'
    neg_target_flag = 'not_reached'
    first_run_flag = 0
    df_reached_targets = pd.DataFrame()
    tick_list = []

    
    
    r = pricing.PricingStream(accountID=accountID, params=params)
    rv = api.request(r)

    start_time = time.time()

    for i, resp in tqdm(enumerate(rv)):

        if i < num: # Check if we are within the required number of price iterations               
            resp_type = resp['type']       

            if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
                pass
                #print(resp_type)

            elif resp_type == 'PRICE': # Check whether it is a price response                 
                date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
                sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                      
                tick_list.append(tick_price)

                if first_run_flag == 0:
                    positive_targets, negative_targets = get_targets(tick_price, target_num)
                    first_run_flag = 1
                    for j in range(target_num):
                        df_reached_targets = df_reached_targets.append(reached_targets, ignore_index = True)
                        df_reached_targets.loc[df_reached_targets.index[j], 'target_num'] = j+1
                        df_reached_targets.loc[df_reached_targets.index[j], 'start_price'] = tick_price
                        df_reached_targets.loc[df_reached_targets.index[j], 'positive_val'] = positive_targets[j]
                        df_reached_targets.loc[df_reached_targets.index[j], 'negative_val'] = negative_targets[j]

                for k in range(target_num):
                    if tick_price >= positive_targets[k]:
                        df_reached_targets.loc[df_reached_targets.index[k], 'positive'] += 1
                        if pos_target_flag == 'not_reached':
                            end_time = time.time()    
                            duration = calc_duration(start_time, end_time)
                            df_reached_targets.loc[df_reached_targets.index[k], 'p_duration'] = duration
                            pos_target_flag = 'reached'

                    if tick_price <= negative_targets[k]:
                        df_reached_targets.loc[df_reached_targets.index[k], 'negative'] += 1            
                        if neg_target_flag == 'not_reached':
                            end_time = time.time()    
                            duration = calc_duration(start_time, end_time)
                            df_reached_targets.loc[df_reached_targets.index[k], 'n_duration'] = duration
                            neg_target_flag = 'reached'

        else: # Crossed the required number of price iterations
            try:
                r.terminate(message = "maxrecs records received")
            except:
                pass

    df_reached_targets =  df_reached_targets[['start_price', 'target_num', 'positive_val', 'negative_val','positive', 'negative','p_duration','n_duration']]
    direction = get_direction(df_reached_targets, target_num, min_count)

    #winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC) 
    return(direction, df_reached_targets, tick_list)

In [7]:
def make_order(accountID, stop_price, instrument, units):
    stopLossOnFill = StopLossDetails(price=stop_price)

    ordr = MarketOrderRequest(
        instrument = instrument,
        units=units,
        stopLossOnFill=stopLossOnFill.data)

    r = orders.OrderCreate(accountID, data=ordr.data)
    rv = api.request(r)
    return(rv)

In [8]:
def close_order(accountID, order_type, instrument):
    data_long = {"longUnits": "ALL"}
    data_short = {"shortUnits": "ALL"}
    
    if order_type == 'long':
        data = data_long
    elif order_type == 'short':
        data = data_short
        
    r = positions.PositionClose(accountID=accountID,
                                instrument=instrument,
                                data=data)
    rv = api.request(r)
    return(rv)

In [9]:
def run_order_engine(instrument, direction): 
    params = {'instruments': instrument}
    pip = 10**-4
    r = pricing.PricingStream(accountID=accountID, params=params)
    rv = api.request(r)
    order_flag = 'not_ordered'
    target = 2 * pip
    
    for i, resp in tqdm(enumerate(rv)):
        
        open_positions_r = positions.OpenPositions(accountID=accountID)
        open_positions_rv = api.request(open_positions_r)
        open_positions = len(open_positions_rv['positions'])        
        
        resp_type = resp['type']       
        
        if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
            pass
        
        elif resp_type == 'PRICE' and order_flag == 'not_ordered': # Check if we are yet to make the order               
            date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
            sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                      

            if direction == 'positive':
                order_type = 'long'
                units = +1
                stop_price = sell_price - (3 * pip)
                make_order_log = make_order(accountID, stop_price, instrument, units)
                order_flag = 'ordered'

            elif direction == 'negative':
                order_type = 'short'
                units = -1                    
                stop_price = buy_price + (3 * pip)
                make_order_log = make_order(accountID, stop_price, instrument, units)                    
                order_flag = 'ordered'

        elif resp_type == 'PRICE' and order_flag == 'ordered': # Check if we have made the order
            sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                                      

            if order_type == 'long':     
                ordered_sell_price = float(make_order_log['orderFillTransaction']['fullPrice']['bids'][0]['price'])
                profit = sell_price - ordered_sell_price                
                if profit >= target:
                    close_order_log = close_order(accountID, order_type ,instrument)
                    order_flag = 'closed'
                    
            if order_type == 'short':     
                ordered_buy_price = float(make_order_log['orderFillTransaction']['fullPrice']['asks'][0]['price'])
                profit = ordered_buy_price - buy_price
                if profit >= target:
                    close_order_log = close_order(accountID, order_type ,instrument)
                    order_flag = 'closed'

                    
        elif resp_type == 'PRICE' and (order_flag == 'closed' or open_positions == 0): # Check if we have made the order
            try:
                r.terminate(message = "")
            except:
                pass

    return(make_order_log, close_order_log)

In [10]:
def get_output_data(make_order_log, close_order_log):
    output_data = {'date':[],
                   'time':[],
                   'instrument':[],
                   'units':[],
                   'profit_pips':[],
                   'orderID':[]
                  }

    make_dict_key = list(make_order_log.keys())[1]
    close_dict_key= list(close_order_log.keys())[1]
    dt, tm, _ = get_date_time(close_order_log[close_dict_key])
    uni = int(make_order_log[make_dict_key]['units'])
    if uni > 0:
        output_data['ordr_type'] = 'long'
    else:
        output_data['ordr_type'] = 'short'

    output_data['date'].append(dt)  
    output_data['time'].append(tm)
    output_data['instrument'].append(close_order_log[close_dict_key]['instrument'])
    output_data['units'].append(uni)
    output_data['profit_pips'].append(close_order_log[close_dict_key]['pl'])
    output_data['orderID'].append(close_order_log[close_dict_key]['orderID'])

    output_df = pd.DataFrame()
    output_df = pd.DataFrame.from_dict(output_data)
    output_df = output_df[['date', 'time', 'instrument','ordr_type','units', 'profit_pips', 'orderID']]
    output_df
    return(output_df)

## API Setup

Read from config file

In [11]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

## Code Engine

In [ ]:
instrument="EUR_USD"
num = 3 * 60
target_num = 3
min_count = 10

direction, df_reached_targets, tick_list = run_direction_engine(instrument, num, target_num, min_count)
print(f'Direction : {direction}')
      
if direction != 'no_direction':
    make_order_log, close_order_log = run_order_engine(instrument, direction)
else:
    print('no_direction : Run again')
    
output_df = get_output_data(make_order_log, close_order_log)

#output_df.to_csv('data/output_log.csv', mode='a')
output_df.to_csv('data/output_log.csv', mode='a', header=False)

150it [01:33,  1.86it/s]